Define a DQN Agent

In [1]:
import random
import gymnasium
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import os

Hyperparameters

In [2]:
env = gymnasium.make('CartPole-v1', render_mode="rgb_array")
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
batch_size = 32
n_episodes = 1000
output_dir = 'model_output/cartpole/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

A deep Q-Learning agent

In [3]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001
        self.model = self._build_model()
        
    def _build_model(self):
        model = Sequential()
        model.add(Dense(32, activation='relu',
                        input_dim=self.state_size))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(learning_rate=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def train(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward #if done
            if not done:
                target = (reward +
                          self.gamma *
                          np.amax(self.model.predict(next_state)))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def save(self, name):
            self.model.save_weights(name)
    
    def load(self, name):
            self.model.load_weights(name)

Agent interaction with OpenAI Gym

In [4]:
agent = DQNAgent(state_size, action_size)

In [ ]:
for e in range(n_episodes):
    state = env.reset()
    state = np.reshape(state[0], [1, state_size])
    done = False
    time = 0
    while not done:
        #env.render()
        action = agent.act(state)
        observation, reward, terminated, info, _ = env.step(action)
        
        done = terminated
        reward = reward if not done else -10
        next_state = np.reshape(observation, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state

        if done:
            print("episode: {}/{}, score: {}, e: {:.2}"
                  .format(e, n_episodes-1, time, agent.epsilon))
        time += 1
    if len(agent.memory) > batch_size:
        agent.train(batch_size)
    if e % 50 == 0:
        agent.save(output_dir + "weights_"
                   + '{:04d}'.format(e) + ".hdf5")

1/1 [==============================] - 0s 17ms/step
episode: 0/999, score: 18, e: 0.84
1/1 [==============================] - 0s 16ms/step
episode: 1/999, score: 22, e: 0.83
1/1 [==============================] - 0s 16ms/step
episode: 2/999, score: 17, e: 0.83
1/1 [==============================] - 0s 16ms/step
episode: 3/999, score: 24, e: 0.83
1/1 [==============================] - 0s 18ms/step
episode: 4/999, score: 32, e: 0.82
1/1 [==============================] - 0s 16ms/step
episode: 5/999, score: 11, e: 0.82
1/1 [==============================] - 0s 16ms/step
episode: 6/999, score: 11, e: 0.81
1/1 [==============================] - 0s 12ms/step
episode: 7/999, score: 27, e: 0.81
1/1 [==============================] - 0s 32ms/step
episode: 8/999, score: 10, e: 0.81
1/1 [==============================] - 0s 32ms/step
episode: 9/999, score: 18, e: 0.8
1/1 [==============================] - 0s 16ms/step
episode: 10/999, score: 17, e: 0.8
1/1 [==============================] - 0s 16